In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,24482
2,Huelva,Confirmados PDIA 14 días,546
3,Huelva,Tasa PDIA 14 días,"106,3974901104897"
4,Huelva,Confirmados PDIA 7 días,204
5,Huelva,Total Confirmados,24709
6,Huelva,Curados,15868
7,Huelva,Fallecidos,284


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  24482.0


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  6534.0


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 3060 personas en los últimos 7 días 

Un positivo PCR cada 946 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,24482.0,204.0,546.0,513170.0,39.752908,106.397490,68.0
Huelva-Costa,14937.0,124.0,342.0,289548.0,42.825369,118.115131,49.0
Condado-Campiña,6822.0,71.0,183.0,156231.0,45.445526,117.134244,26.0
Huelva (capital),6534.0,47.0,152.0,143837.0,32.675876,105.675174,16.0
Cartaya,934.0,20.0,52.0,20083.0,99.586715,258.925459,11.0
Almonte,847.0,16.0,49.0,24507.0,65.287469,199.942873,9.0
Palma del Condado (La),969.0,8.0,11.0,10801.0,74.067216,101.842422,5.0
Aljaraque,922.0,8.0,14.0,21474.0,37.254354,65.195120,4.0
Moguer,820.0,15.0,32.0,21867.0,68.596515,146.339233,4.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Villablanca,174.0,6.0,10.0,2885.0,207.972270,346.620451,2.0
Villanueva de los Castillejos,150.0,3.0,9.0,2825.0,106.194690,318.584071,1.0
Cartaya,934.0,20.0,52.0,20083.0,99.586715,258.925459,11.0
Escacena del Campo,137.0,1.0,5.0,2287.0,43.725404,218.627022,-1.0
Bollullos Par del Condado,605.0,11.0,29.0,14387.0,76.457913,201.570863,3.0
Almonte,847.0,16.0,49.0,24507.0,65.287469,199.942873,9.0
Villalba del Alcor,443.0,5.0,6.0,3366.0,148.544266,178.253119,1.0
Rosal de la Frontera,25.0,3.0,3.0,1697.0,176.782557,176.782557,2.0
Lepe,1986.0,20.0,43.0,27880.0,71.736011,154.232425,4.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Gibraleón,509.0,1.0,7.0,12737.0,7.851142,54.957996,1.0,0.142857
Escacena del Campo,137.0,1.0,5.0,2287.0,43.725404,218.627022,-1.0,0.200000
Isla Cristina,1978.0,5.0,21.0,21393.0,23.372131,98.162950,3.0,0.238095
Ayamonte,1184.0,5.0,18.0,21104.0,23.692191,85.291888,1.0,0.277778
Huelva (capital),6534.0,47.0,152.0,143837.0,32.675876,105.675174,16.0,0.309211
Almonte,847.0,16.0,49.0,24507.0,65.287469,199.942873,9.0,0.326531
Villanueva de los Castillejos,150.0,3.0,9.0,2825.0,106.194690,318.584071,1.0,0.333333
Bonares,178.0,1.0,3.0,6060.0,16.501650,49.504950,0.0,0.333333
Punta Umbría,655.0,6.0,18.0,15355.0,39.075220,117.225659,3.0,0.333333
